In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from ipywidgets import Dropdown

In [2]:
# モデルの選択（2B モデルか 7B モデルかを選択）
model_options = ["Qwen/Qwen2-VL-2B-Instruct", "Qwen/Qwen2-VL-7B-Instruct"]
model_dropdown = Dropdown(options=model_options, value=model_options[0], description='Model:')

# ドロップダウンを表示
display(model_dropdown)

def get_model_id(change):
    global model_id
    model_id = change.new

# 値が変更されたときにget_model_id関数を呼び出す
model_dropdown.observe(get_model_id, names='value')

model_id = model_dropdown.value


Dropdown(description='Model:', options=('Qwen/Qwen2-VL-2B-Instruct', 'Qwen/Qwen2-VL-7B-Instruct'), value='Qwen…

In [3]:
# モデルをロード
print(f"loading {model_id}...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id, torch_dtype="auto", device_map="auto"
)

loading Qwen/Qwen2-VL-2B-Instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [4]:
# プロセッサの初期化
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

In [5]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "chat_with_ai.jpg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

In [6]:
# プロンプトの準備
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>
<|im_start|>assistant



In [7]:
# イメージの準備
image_inputs, video_inputs = process_vision_info(messages)

In [8]:
# モデル入力の準備
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [9]:
# 推論：出力の生成
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

d:\lab\qwen2-vl\qwen2-vl-2b-instruct\.venv\Lib\site-packages\transformers\models\qwen2_vl\modeling_qwen2_vl.py:350: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = F.scaled_dot_product_attention(q, k, v, attention_mask, dropout_p=0.0)


['The image depicts a futuristic scene where a humanoid robot is interacting with a human figure. The robot, which appears to be made of glowing, pixelated particles, is seated and facing the human figure. The human figure is dressed in a formal outfit and is sitting on a bench, looking up at the robot with a smile. The background features a modern, illuminated corridor with tall buildings and bright lights, suggesting a high-tech environment. The word "Chat" is prominently displayed in the foreground, indicating the theme or context of the image. The overall atmosphere is one of advanced technology and human interaction.']
